# 🏆 [99.47% 準確率] 透過微調 ResNet50 實現植物病害辨識
# 🏆 [99.47% Accuracy] Fine-tuning ResNet50 for Plant Disease Classification

## 專案簡介 (中文)
本專案旨在重現一個在 PlantVillage 資料集上達到 **99.47%** 測試集準確率的高精度植物病害辨識模型。這個成果是透過對一個在 ImageNet 上預訓練的 **ResNet50** 模型，採用**微調 (Fine-tuning)** 策略而達成的。
---
## Project Introduction (English)
The purpose of this project is to reproduce a high-accuracy plant disease classification model that achieves **99.47% accuracy** on the PlantVillage dataset's hold-out test set. This state-of-the-art (SOTA) result was achieved by applying a **Fine-tuning** strategy to a **ResNet50** model pre-trained on ImageNet.

### 步驟 1：導入函式庫與設定路徑
### Step 1: Import Libraries and Setup Paths

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.models as models # 導入 models 模組 / Import the models module
from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print("All libraries imported successfully!")

PLANTDISEASE_DATASET_DIR = '/kaggle/input/plantdisease' 
ASSETS_DIR = '/kaggle/input/plantdisease-finetuned-resnet50-99-47-assets' 

BASE_ASSETS_PATH = os.path.join(ASSETS_DIR, 'plantdisease')
MODEL_WEIGHTS_PATH = os.path.join(BASE_ASSETS_PATH, 'output', 'train_ResNet50_FineTune_output', 'resnet50_finetune_model_fold_3_best.pth')
TEST_CSV_PATH = os.path.join(BASE_ASSETS_PATH, 'test_set.csv')
JSON_MAPPING_PATH = os.path.join(BASE_ASSETS_PATH, 'label_mapping.json')

print("Paths configured:")
print(f"- Model Path: {MODEL_WEIGHTS_PATH}")
print(f"- Test CSV Path: {TEST_CSV_PATH}")

### 步驟 2：定義資料集與模型架構
### Step 2: Define Dataset Class and Model Architecture

In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 32
NUM_CLASSES = 15

class PlantDiseaseDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.df = dataframe
        self.transform = transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        win_relative_path = self.df.iloc[idx]['filepath']
        nix_relative_path = win_relative_path.replace('\\', '/')
        img_path = os.path.join(PLANTDISEASE_DATASET_DIR, nix_relative_path)
        label = self.df.iloc[idx]['label_idx']
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label, dtype=torch.long)

def get_model(num_classes=15):
    # *** 核心修正點：使用新的 API 以消除警告 ***
    # *** Core Fix: Use the new API to eliminate warnings ***
    model = models.resnet50(weights=None) 
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
    return model

print("Helper class and function defined with updated API.")

### 步驟 3：執行評估
### Step 3: Run Evaluation

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

test_df = pd.read_csv(TEST_CSV_PATH)

val_transforms = transforms.Compose([
    transforms.Resize(256), transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_dataset = PlantDiseaseDataset(test_df, transform=val_transforms)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

model = get_model(num_classes=NUM_CLASSES).to(device)
model.load_state_dict(torch.load(MODEL_WEIGHTS_PATH, map_location=device))
model.eval()

all_preds, all_labels = [], []
with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc="Predicting on Test Set"):
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print("Prediction complete!")

### 步驟 4：結果分析與視覺化
### Step 4: Analyze and Visualize Results

In [ ]:
print("="*50)
print("       REPRODUCED EVALUATION RESULTS ON KAGGLE       ")
print("="*50)

accuracy = accuracy_score(all_labels, all_preds)
print(f"Overall Test Accuracy: {accuracy:.4f}\n")

with open(JSON_MAPPING_PATH, 'r') as f:
    label_mapping = {int(k): v for k, v in json.load(f).items()}
class_names = [label_mapping[i] for i in range(NUM_CLASSES)]

print("Classification Report:")
report = classification_report(all_labels, all_preds, target_names=class_names, digits=4)
print(report)

print("\nGenerating confusion matrix...")
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(18, 14))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Label', fontsize=14)
plt.ylabel('True Label', fontsize=14)
plt.title('Confusion Matrix for Fine-tuned ResNet50 (99.47% Accuracy)', fontsize=18)
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.show()